In [14]:
import os
import random
import json
import requests
from operator import itemgetter
from typing import Union, List
from langchain_openai import AzureChatOpenAI
from langchain.agents import AgentExecutor, Tool, create_openai_tools_agent
from langchain_community.chat_message_histories import ChatMessageHistory, CosmosDBChatMessageHistory
from langchain.callbacks.manager import CallbackManager
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import ConfigurableFieldSpec, ConfigurableField
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import JsonOutputToolsParser
from langchain_core.runnables import (
    Runnable,
    RunnableLambda,
    RunnableMap,
    RunnablePassthrough,
)

#custom libraries that we will use later in the app
from common.utils import (
    SQLSearchAgent
)
from common.callbacks import StdOutCallbackHandler
from common.prompts import CUSTOM_CHATBOT_PROMPT 

from dotenv import load_dotenv
load_dotenv("credentials.env")

from IPython.display import Markdown, HTML, display 

def printmd(string):
    display(Markdown(string))

In [15]:
cb_handler = StdOutCallbackHandler()
cb_manager = CallbackManager(handlers=[cb_handler])

COMPLETION_TOKENS = 2000

# We can run the everything with GPT3.5, but try also GPT4 and see the difference in the quality of responses
# You will notice that GPT3.5 is not as reliable when using multiple sources.

# llm = AzureChatOpenAI(deployment_name=os.environ["GPT4o_DEPLOYMENT_NAME"], 
#                       temperature=0, max_tokens=COMPLETION_TOKENS)

# Uncomment below if you want to see the answers streaming
llm = AzureChatOpenAI(deployment_name=os.environ["GPT4o_DEPLOYMENT_NAME"], temperature=0, max_tokens=COMPLETION_TOKENS, streaming=True, callback_manager=cb_manager, api_version="2024-05-01-preview")
# llm = ChatOllama(model = "llama3.2:latest", max_tokens = 128000, temperature=0, callbacks=cb_manager, disable_streaming = False)

In [16]:
sql_search = SQLSearchAgent(llm=llm, k=10, callback_manager=cb_manager,
                name="sqlsearch",
                description="useful when the questions includes the term: sqlsearch",
                verbose=False)

# Multi Agent

In [17]:
tools = [sql_search]

In [18]:
agent = create_openai_tools_agent(llm, tools, CUSTOM_CHATBOT_PROMPT)

In [19]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

In [20]:
def get_session_history(session_id: str, user_id: str) -> CosmosDBChatMessageHistory:
    cosmos = CosmosDBChatMessageHistory(
        cosmos_endpoint=os.environ['AZURE_COSMOSDB_ENDPOINT'],
        cosmos_database=os.environ['AZURE_COSMOSDB_NAME'],
        cosmos_container=os.environ['AZURE_COSMOSDB_CONTAINER_NAME'],
        connection_string=os.environ['AZURE_COMOSDB_CONNECTION_STRING'],
        session_id=session_id,
        user_id=user_id
        )

    # prepare the cosmosdb instance
    cosmos.prepare_cosmos()
    return cosmos

In [21]:
brain_agent_executor = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="user_id",
            annotation=str,
            name="User ID",
            description="Unique identifier for the user.",
            default="",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="Unique identifier for the conversation.",
            default="",
            is_shared=True,
        ),
    ],
)

In [22]:
# This is where we configure the session id and user id
random_session_id = "session"+ str(random.randint(1, 1000))
ramdom_user_id = "user"+ str(random.randint(1, 1000))

# random_session_id = "session428" 
# ramdom_user_id = "user325"
config={"configurable": {"session_id": random_session_id, "user_id": ramdom_user_id}}
print(random_session_id, ramdom_user_id)

session578 user455


# Further Testing

In [23]:
# printmd(brain_agent_executor.invoke({"question": "Hi"}, 
#                                     config=config)["output"])
# Ensure these variables are defined before invoking the template
from datetime import datetime, timedelta
today = datetime.now().date()
yesterday = today - timedelta(days=1)
day_before_yesterday = today - timedelta(days=2)

# Format dates as strings
target_date_8 = day_before_yesterday.strftime("%Y-%m-%d")
target_date_9 = yesterday.strftime("%Y-%m-%d")
target_date_today = today.strftime("%Y-%m-%d")
inputs = {
    "question": "Hi",
    "history": [],
    "agent_scratchpad": [],
    "target_date_8": target_date_8,
    "target_date_9": target_date_9,
    "target_date_today": target_date_today
}

output = brain_agent_executor.invoke(inputs, config=config)["output"]
printmd(output)

Good morning! Ready for a productive day ahead? Let's get started with your daily standup update! Are you ready to provide your update now, or is there something else you'd like to discuss first?

Good morning! Ready for a productive day ahead? Let's get started with your daily standup update! Are you ready to provide your update now, or is there something else you'd like to discuss first?

In [27]:
# Ensure these variables are defined before invoking the template
from datetime import datetime, timedelta

today = datetime.now().date()
yesterday = today - timedelta(days=1)
day_before_yesterday = today - timedelta(days=2)

# Format dates as strings
target_date_8 = day_before_yesterday.strftime("%Y-%m-%d")
target_date_9 = yesterday.strftime("%Y-%m-%d")
target_date_today = today.strftime("%Y-%m-%d")

inputs = {
    "question": "Give me a summary of the github issues from January 8th",
    "history": [],
    "agent_scratchpad": [],
    "target_date_8": target_date_8,
    "target_date_9": target_date_9,
    "target_date_today": target_date_today
}

output = brain_agent_executor.invoke(inputs, config=config)["output"]
printmd(output)

I don't have access to the GitHub activities from January 8th. My available data starts from January 9th, 2025. If you need information from another date within my range or have any other questions, feel free to ask!

I don't have access to the GitHub activities from January 8th. My available data starts from January 9th, 2025. If you need information from another date within my range or have any other questions, feel free to ask!

In [12]:
printmd(brain_agent_executor.invoke({"question": "Give me a summary of the github issues from 9th January"}, 
                                    config=config)["output"])

KeyError: "Input to ChatPromptTemplate is missing variables {'target_date_9'}.  Expected: ['agent_scratchpad', 'question', 'target_date_9'] Received: ['question', 'history', 'intermediate_steps', 'agent_scratchpad']\nNote: if you intended {target_date_9} to be part of the string and not a variable, please escape it with double curly braces like: '{{target_date_9}}'."

In [13]:
printmd(brain_agent_executor.invoke({"question": "sqlsearch Give me all details of dailypulse"}, 
                                    config=config)["output"])

Tool: sqlsearch
Agent Action: 
Invoking: `sql_db_list_tables` with `{}`



Agent Action: 
Invoking: `sql_db_schema` with `{'table_names': 'dailypulse'}`



The `dailypulse` table in the database has the following structure:

| Column Name     | Data Type        | Constraints     |
|-----------------|------------------|-----------------|
| id              | SERIAL           | NOT NULL, PRIMARY KEY |
| username        | VARCHAR(255)     | NOT NULL        |
| accomplishment  | TEXT             |                 |
| todo            | TEXT             |                 |
| blocker         | TEXT             |                 |
| date            | DATE             | NOT NULL        |

Here are some sample rows from the `dailypulse` table:

| id | username   | accomplishment | todo | blocker | date       |
|----|------------|----------------|------|---------|------------|
| 1  | ashishkj21 | Closed PRs: Initial memory commit and direct message functionality. | Complete DB setup, Address lack 

Here are the details from the `dailypulse` table in the database for your username "ashishkj21":

1. **Date: 2025-01-07**
   - **Accomplishments**: Closed PRs: Initial memory commit and direct message functionality.
   - **Plans**: Complete DB setup, Address lack of descriptive README.
   - **Blockers**: Incomplete DB setup.

2. **Date: 2025-01-09**
   - **Accomplishments**: Successfully merged pull requests related to adding all tools to the github_toolkit.py and implement.
   - **Plans**: Continue working on the database setup for the codebase.
   - **Blockers**: Incomplete database setup.

3. **Date: 2025-01-09**
   - **Accomplishments**: Successfully merged the pull request for "Direct message functionality" and "Initial memory commit".
   - **Plans**: Continue working on the database setup for the codebase, as noted in the recently opened issue. Address...
   - **Blockers**: Getting blocked due to lack of tools for date and time segregation.

If you need more specific information or further details on any of these entries, please let me know!